In [1]:
# Examples
## https://medium.com/@djajafer/multi-class-text-classification-with-keras-and-lstm-4c5525bef592
## BERT for Multiclass
## https://medium.com/@chandaksumit29_15695/bert-for-multi-class-text-classification-12b66a1fc01c

# Dataset
## Imbalance
## https://towardsdatascience.com/machine-learning-multiclass-classification-with-imbalanced-data-set-29f6a177c1a#:~:text=Multi%2Dclass%20classification%20makes%20the,classes%20are%20not%20represented%20equally.
#https://stackoverflow.com/questions/61969783/huggingface-bert-showing-poor-accuracy-f1-score-pytorch
#Comparatively Finetunign BERT https://github.com/uzaymacar/comparatively-finetuning-bert
# Finetuning BERT for sentiment analysis
## https://skimai.com/fine-tuning-bert-for-sentiment-analysis/

# Training
## Batch Normalization
## https://towardsdatascience.com/batch-normalization-in-neural-networks-1ac91516821c
## Batchsize
## https://dmitry.ai/t/topic/100

# Loss Function
## F1-Weighted Loss Function: https://stackoverflow.com/questions/59sam63911/how-to-write-a-custom-f1-loss-function-with-weighted-average-for-keras
## How to choose loss function: https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/
## Loss function for f1 https://www.kaggle.com/rejpalcz/best-loss-function-for-f1-score-metric
## Cross Entropy: https://stats.stackexchange.com/questions/272754/how-do-you-interpret-the-cross-entropy-value
## https://stats.stackexchange.com/questions/207794/what-loss-function-for-multi-class-multi-label-classification-tasks-in-neural-n

# Problens while finetuning
# https://github.com/google-research/bert/issues/663
## High Loss Good accuracy
## https://stats.stackexchange.com/questions/258166/good-accuracy-despite-high-loss-value/281651#281651
## High Accuracy Low F1
## https://www.quora.com/How-do-I-interpret-a-high-accuracy-82-44-but-an-extremely-low-F1-score-0-259-in-my-deep-learning-model-How-do-I-go-about-improving-the-F1-score
## How is it possible that validation loss is increasing while validation accuracy is increasing as well
## https://stats.stackexchange.com/questions/282160/how-is-it-possible-that-validation-loss-is-increasing-while-validation-accuracy
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Select the Runtime > "Change runtime type" menu to enable a GPU accelerator, ')
  print('and then re-execute this cell.')
else:
  print(gpu_info)

# memory footprint support libraries/code
!ln -sf /opt/bin/nvidia-smi /usr/bin/nvidia-smi
!pip install gputil
!pip install psutil
!pip install humanize    
import psutil
import humanize
import os
import GPUtil as GPU
GPUs = GPU.getGPUs()
# XXX: only one GPU on Colab and isn't guaranteed
gpu = GPUs[0]
#gpu2 = GPUs[1]

def printm():
  process = psutil.Process(os.getpid())
  print("Gen RAM Free: " + humanize.naturalsize( psutil.virtual_memory().available ), " I Proc size: " + humanize.naturalsize( process.memory_info().rss))
  print('GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB'.format(gpu.memoryFree, gpu.memoryUsed, gpu.memoryUtil*100, gpu.memoryTotal))
#  print('GPU RAM Free: {0:.0f}MB | Used: {1:.0f}MB | Util {2:3.0f}% | Total {3:.0f}MB'.format(gpu2.memoryFree, gpu2.memoryUsed, gpu2.memoryUtil*100, gpu2.memoryTotal))
    
printm()

Sat May 15 13:08:29 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 450.80.02    Driver Version: 450.80.02    CUDA Version: 11.0     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla K80           Off  | 00000000:00:04.0 Off |                    0 |
| N/A   49C    P0    67W / 149W |      0MiB / 11441MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  Tesla K80           Off  | 00000000:00:05.0 Off |                    0 |
| N/A   

In [2]:
#sudo kill -9 1748

In [3]:
#!sudo fuser -v /dev/nvidia*

In [4]:
%pip install transformers
%pip install datasets
!pip install focal_loss
import pandas as pd
import pyarrow
if int(pyarrow.__version__.split('.')[1]) < 16 and int(pyarrow.__version__.split('.')[0]) == 0:
    import os

    
    os.kill(os.getpid(), 9)
from datasets import load_dataset, Dataset
from datasets import ClassLabel, Features, Value
# Tensorflow classification with imbalanced data https://www.tensorflow.org/tutorials/structured_data/imbalanced_data#calculate_class_weights
# How to choose loss function https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/
#https://colab.research.google.com/drive/1kEg0SnYNtw_IJwu_kl5y3qRVs-BKBmNO#scrollTo=W64yQeCyl8sC
#https://github.com/NadirEM/nlp-notebooks/blob/master/Fine_tune_ALBERT_sentence_pair_classification.ipynb
#https://github.com/abhimishra91/transformers-tutorials/blob/master/transformers_multiclass_classification.ipynb
#https://huggingface.co/transformers/master/custom_datasets.html
#https://huggingface.co/transformers/custom_datasets.html#sequence-classification-with-imdb-reviews

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [5]:
max_length=256
model_name='roberta-large'


In [6]:

from sklearn.model_selection import train_test_split

def load_data(file_path):
    loaded_data = pd.read_csv(file_path, sep='\t', header=None)
    return loaded_data


def reverse_multihot(val):
    encoding = {0:'none', 1:'exact', 2:'broader', 3:'narrower', 4:'related'}
    return encoding[val]

def split_data(data, testset_ratio):
    train_df, remaining = train_test_split(data, random_state=42, train_size=0.75, stratify=data.labels.values)
    return train_df, remaining
    #data = data.sample(frac=1)
    #f = int(len(data) * testset_ratio)
    #return data[f:], data[:f]

def multihot(val):
    encoding = {'none':0, 'exact':1, 'broader':2, 'narrower':3, 'related':4}
    return encoding[val]


def encode(data):
  return tokenizer(data['text_a'], data['text_b'], truncation=True, max_length=max_length, padding='max_length')


In [7]:

folder = 'english_nuig.tsv'
en_data = load_data(folder)
en_data = en_data[[2,3,4]]
en_data.columns =  ['text_a','text_b','labels']
en_data['labels'].value_counts()
#en_data['labels'] = en_data['labels'].map(lambda val: multihot(val))

none        7137
exact        800
narrower     310
related       51
broader       39
Name: labels, dtype: int64

In [8]:
test_file = 'english_nuig_reference.tsv'
test_data_orig = load_data(test_file)
test_data = test_data_orig[[2,3,4]]
test_data.columns = ['text_a', 'text_b', 'labels']

In [9]:
label_list = ['broader','narrower','exact','related','none']
en_data['labels'].apply(label_list.index)
test_data['labels'].apply(label_list.index)

0      3
1      2
2      3
3      4
4      3
      ..
539    4
540    4
541    3
542    4
543    2
Name: labels, Length: 544, dtype: int64

In [10]:
trainset, valset = split_data(en_data, 0.2)
trainset.shape, valset.shape
trainset.text_a.values

array(['rouse somebody from sleep with a call',
       "insist on having one's opinions and rights recognized",
       'give a workout to', ...,
       'a layer of material that encloses space', 'cause annoyance in',
       'loss resulting from failure of a debt to be paid'], dtype=object)

In [11]:
from sklearn import preprocessing
import numpy


In [12]:
le = preprocessing.LabelEncoder()
le.fit(en_data['labels'].to_list())

labels = le.classes_
labelMap = {v: k for v, k in enumerate(labels)}
    
print(labelMap)

def onehot_encode(data, encoder):

  integer_encoded = encoder.transform(data['labels'].to_list())
  onehot_encoder = preprocessing.OneHotEncoder(sparse=False)
  integer_encoded = integer_encoded.reshape(len(integer_encoded), 1)
  
  return onehot_encoder.fit_transform(integer_encoded)

{0: 'broader', 1: 'exact', 2: 'narrower', 3: 'none', 4: 'related'}


In [13]:
onehot_encoded_train = onehot_encode(trainset, le)
onehot_encoded_val = onehot_encode(valset, le)
onehot_encoded_test = onehot_encode(test_data, le)

In [14]:
trainset

,text_a,text_b,labels
2199,rouse somebody from sleep with a call,to regard or characterize as of a certain kind...,none
3779,insist on having one's opinions and rights rec...,"to maintain or defend, as a cause or a claim, ...",exact
3909,give a workout to,to exert one's self for a purpose; to put fort...,none
1881,belonging to or characteristic of the nobility...,"having the color of the clear sky, or a hue re...",none
3466,insert or adjust several objects or people,to be proper or becoming.,none
...,...,...,...
5587,the sound made by a gentle blow,"a light, quik blow or stroke with the fingers ...",none
7899,teach or refine to be discriminative in taste ...,"to draw by persuasion, artifice, or the like; ...",none
2665,a layer of material that encloses space,a kind of knot often used at the end of a rope...,none
633,cause annoyance in,to be agitated; to be in violent commotion; to...,none


In [15]:
onehot_encoded_train

array([[0., 0., 0., 1., 0.],
       [0., 1., 0., 0., 0.],
       [0., 0., 0., 1., 0.],
       ...,
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.],
       [0., 0., 0., 1., 0.]])

In [16]:
val_y = le.inverse_transform([numpy.argmax(onehot_encoded_val[2, :])])
val_y

array(['none'], dtype='<U8')

In [17]:
import numpy as np
from sklearn.utils.class_weight import compute_class_weight, compute_sample_weight
y_integers = np.argmax(onehot_encoded_train, axis=1)
class_weights = compute_class_weight('balanced', np.unique(y_integers), y_integers)
d_class_weights = dict(enumerate(class_weights))
sample_weights = compute_sample_weight('balanced', onehot_encoded_train)
sample_weights_val = compute_sample_weight('balanced', onehot_encoded_val)

/opt/conda/lib/python3.7/site-packages/sklearn/utils/validation.py:72: FutureWarning: Pass classes=[0 1 2 3 4], y=[3 1 3 ... 3 3 3] as keyword args. From version 1.0 (renaming of 0.25) passing these as positional arguments will result in an error
  "will result in an error", FutureWarning)


In [18]:
set(sample_weights)

{0.04239661830954378,
 2.3749568058166997,
 6.512882377104106,
 41.22801874886851,
 54.10116480927345}

In [19]:
train_a = trainset['text_a'].to_list()
train_b = trainset['text_b'].to_list()
train_labels = onehot_encoded_train

val_a = valset['text_a'].to_list()
val_b = valset['text_b'].to_list()
val_labels = onehot_encoded_val

test_a = test_data['text_a'].to_list()
test_b = test_data['text_b'].to_list()
test_labels = onehot_encoded_test

In [20]:
from transformers import RobertaTokenizerFast
tokenizer = RobertaTokenizerFast.from_pretrained(model_name)


In [21]:
train_encodings = tokenizer(train_a, train_b, truncation=True, padding='longest',max_length=max_length, return_tensors='tf')
val_encodings = tokenizer(val_a, val_b, truncation=True, padding='longest', max_length=max_length, return_tensors='tf')
test_encodings = tokenizer(test_a, test_b, truncation=True,padding='longest', max_length=max_length,return_tensors='tf')

In [22]:

import tensorflow as tf

train_dataset = tf.data.Dataset.from_tensor_slices((
    dict(train_encodings),
    train_labels,
    sample_weights
))
val_dataset = tf.data.Dataset.from_tensor_slices((
    dict(val_encodings),
    val_labels,
    sample_weights_val
))
test_dataset = tf.data.Dataset.from_tensor_slices((
    dict(test_encodings),
    test_labels
))
# Oversampling
# https://github.com/tensorflow/tensorflow/issues/14451
#https://keras.io/api/models/model_training_apis/  
# https://stackoverflow.com/questions/53653485/training-with-tf-data-api-and-sample-weights

In [23]:
train_dataset.element_spec
#!pip install tensorflow_datasets

({'input_ids': TensorSpec(shape=(256,), dtype=tf.int32, name=None),
  'attention_mask': TensorSpec(shape=(256,), dtype=tf.int32, name=None)},
 TensorSpec(shape=(5,), dtype=tf.float64, name=None),
 TensorSpec(shape=(), dtype=tf.float64, name=None))

In [24]:
#import tensorflow_datasets as tfds
#for pair, label in tfds.as_numpy(train_dataset.take(1)):
#    print('pair', pair, label)

In [25]:
train_labels[0]
le.inverse_transform([0])
!pip install wandb
!wandb login 8d2052d6ffc37b0b4398c538a9604fe6b18e0cf9


wandb: Appending key for api.wandb.ai to your netrc file: /home/jupyter/.netrc


In [26]:
!pip install focal_loss

In [27]:
printm()

Gen RAM Free: 12.6 GB  I Proc size: 2.2 GB
GPU RAM Free: 11441MB | Used: 0MB | Util   0% | Total 11441MB


In [28]:
#!pip install Keras==2.4.3
!pip install tensorflow-addons==0.8.3


In [29]:
#import keras
import tensorflow_addons as tfa
#from tfa.losses import SigmoidFocalCrossEntropy
#https://www.tensorflow.org/guide/keras/custom_callback#an_overview_of_callback_methods
class LossAndErrorPrintingCallback(tf.keras.callbacks.Callback):
    def on_train_batch_end(self, batch, logs=None):
        print("For batch {}, loss is {:7.2f}.".format(batch, logs["loss"]))

    def on_test_batch_end(self, batch, logs=None):
        print("For batch {}, loss is {:7.2f}.".format(batch, logs["loss"]))

    def on_epoch_end(self, epoch, logs=None):
        print(
            "The average loss for epoch {} is {:7.2f} ".format(
                epoch, logs["loss"]
            )
        )

class LRLogger(tf.keras.callbacks.Callback):
    """Learning rate scheduler which sets the learning rate according to schedule.

  Arguments:
      schedule: a function that takes an epoch index
          (integer, indexed from 0) and current learning rate
          as inputs and returns a new learning rate as output (float).
  """

    def __init__(self):
        super(LRLogger, self).__init__()

    def on_epoch_begin(self, epoch, logs=None):
        if not hasattr(self.model.optimizer, "lr"):
            raise ValueError('Optimizer must have a "lr" attribute.')
        # Get the current learning rate from model's optimizer.
        #lr = float(tf.keras.backend.get_value(self.model.optimizer.learning_rate))
        lr = self.model.optimizer.learning_rate(self.model.optimizer.iterations)
        print('learning rate: '+str(lr) + 'iteration: ' + str(self.model.optimizer.iterations) )
        wandb.log({'epoch': epoch, 'learning_rate': lr})
        # Call schedule function to get the scheduled learning rate.
        #scheduled_lr = self.schedule(epoch, lr)
        # Set the value back to the optimizer before this epoch starts
        #tf.keras.backend.set_value(self.model.optimizer.lr, scheduled_lr)
        #print("\nEpoch %05d: Learning rate is %6.4f." % (epoch, scheduled_lr))

In [30]:
#GPU.getGPUs()

In [35]:

from wandb.keras import WandbCallback

import wandb
import gc



#from transformers import TFDistilBertForSequenceClassification
from transformers import TFRobertaForSequenceClassification,AdamWeightDecay
from tensorflow.keras.losses import CategoricalCrossentropy
from tensorflow.keras.metrics import CategoricalAccuracy
from tensorflow_addons.metrics import F1Score
from tensorflow.keras.callbacks import ReduceLROnPlateau

seed=777
tf.random.set_seed(seed)

#wandb.config.globalseed=seed
#wandb.config.nr_gpus=len(GPU.getGPUs())
#wandb.config.lr = 4e-5
#wandb.config.weight_decay_rate=0.2
#wandb.config.epoch=10
#wandb.config.batch=32
#wandb.config.max_length=max_length
#wandb.config.lr_scheduler='ReduceLROnPlateau'
#wandb.config.seed=seed
#wandb.config.model=model_name





In [32]:
reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.2,
                              patience=3, min_lr=0)

# Define multi-gpu strategy 
mirrored_strategy = tf.distribute.MirroredStrategy()
# Update batch size value
#wandb.config.batch *= mirrored_strategy.num_replicas_in_sync
# Create strategy scope to perform training

# Transformer and Batchsize
#https://dmitry.ai/t/topic/100/2


class RemoveGarbaseCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs=None):
    gc.collect()


INFO:tensorflow:Using MirroredStrategy with devices ('/job:localhost/replica:0/task:0/device:GPU:0', '/job:localhost/replica:0/task:0/device:GPU:1', '/job:localhost/replica:0/task:0/device:GPU:2', '/job:localhost/replica:0/task:0/device:GPU:3', '/job:localhost/replica:0/task:0/device:GPU:4', '/job:localhost/replica:0/task:0/device:GPU:5', '/job:localhost/replica:0/task:0/device:GPU:6', '/job:localhost/replica:0/task:0/device:GPU:7')


In [33]:
class_weights_dict={}
for i in range(len(class_weights)):
    class_weights_dict[i]=class_weights[i]
    

In [34]:
from transformers.optimization_tf import WarmUp


In [36]:
from tensorflow_addons.losses import SigmoidFocalCrossEntropy
from transformers.optimization_tf import create_optimizer

def train():
    
    hyperparameter_defaults = dict(
        nr_gpus=len(GPU.getGPUs()),
        lr=5e-6,
        model=model_name,
        globalseed=seed,
        weight_decay_rate=0.1,
        epoch=10,
        batch=16,
        max_length=max_length,
        seed=seed
    )
    wandb.init(project="mwsa_transformers", config=hyperparameter_defaults)
    config = wandb.config
    

    #https://www.tensorflow.org/official_models/fine_tuning_bert
    train_data_size = len(train_labels)
    steps_per_epoch = int(train_data_size/config.batch)
    num_train_steps = steps_per_epoch*config.epoch
    warmup_steps = int(config.epoch*train_data_size*0.1/config.batch)
    
    min_lr_ratio: float = 0.0
    power: float = 1.0
    lr_schedule = tf.keras.optimizers.schedules.PolynomialDecay(
        initial_learning_rate=config.lr,
        decay_steps=num_train_steps - warmup_steps,
        end_learning_rate=config.lr * min_lr_ratio,
        power=power,
    )
      
    warmup_scheduler = WarmUp(config.lr, lr_schedule, warmup_steps=warmup_steps)
    
    with mirrored_strategy.scope():
        #https://www.tensorflow.org/api_docs/python/tf/keras/losses/CategoricalCrossentropy
        #loss = {'issue': CategoricalCrossentropy(from_logits = False), 'product': CategoricalCrossentropy(from_logits = True)}
        loss = SigmoidFocalCrossEntropy(from_logits=True, alpha=0.25, gamma=2.0)
        metric = [CategoricalAccuracy(), F1Score(num_classes=5, average='weighted')]
        # How to choose loss functions https://machinelearningmastery.com/how-to-choose-loss-functions-when-training-deep-learning-neural-networks/
        #https://stackoverflow.com/questions/59305514/tensorflow-how-to-use-tf-keras-metrics-in-multiclass-classification
        #https://towardsdatascience.com/keras-accuracy-metrics-8572eb479ec7#:~:text=Categorical%20Accuracy%20calculates%20the%20percentage,yTrue%2C%20it%20is%20considered%20accurate.
        #optimizer = tf.keras.optimizers.Adam(learning_rate=5e-5)
        #optimizer = AdamWeightDecay(learning_rate=0.01, weight_decay_rate=0.1)#silvery-plasma-24
        optimizer = AdamWeightDecay(learning_rate=warmup_scheduler, weight_decay_rate=wandb.config.weight_decay_rate)
        #optimizer = create_optimizer(init_lr=wandb.config.lr, num_train_steps=num_train_steps, num_warmup_steps=warmup_steps)
        model = TFRobertaForSequenceClassification.from_pretrained(model_name, num_labels=5, return_dict=True)
        #model.summary()
        model.config.gradient_checkpointing=True

        #https://keras.io/api/models/model_training_apis/
        model.compile(optimizer=optimizer, loss=loss, metrics=metric, weighted_metrics=metric) # can also use any keras loss fn
        history = model.fit(train_dataset.shuffle(1000,seed=wandb.config.seed).batch(wandb.config.batch), 
              epochs=wandb.config.epoch,
              validation_data=val_dataset.shuffle(1000,seed=wandb.config.seed).batch(wandb.config.batch), 
                            class_weight=class_weights_dict,
                            callbacks=[WandbCallback(),
                                       reduce_lr,LRLogger(),
                                       RemoveGarbaseCallback()])
        
        return model



# Steps per epoch
# https://datascience.stackexchange.com/questions/47405/what-to-set-in-steps-per-epoch-in-keras-fit-generator

#https://www.tensorflow.org/tutorials/keras/text_classification
# Increasing Loss and accuracy
#Overfitting
#https://datascience.stackexchange.com/questions/76527/overfitting-in-huggingfaces-tfbertforsequenceclassification

In [37]:
sweep_config = {
  "name": "MWSA Sweep",
  "method": "grid",
  "parameters": {
        "lr": {
            "values": [5e-5,5e-4,5e-6]
        },
        "weight_decay_rate":{
            "values": [0.1,0.2,0.3]
        },
        "batch": {
            "values": [32]
        }
  }
}

sweep_id = wandb.sweep(sweep_config)

model = train()
#wandb.agent(sweep_id, train)

Create sweep with ID: g44958cn
Sweep URL: https://wandb.ai/sbyim/uncategorized/sweeps/g44958cn


wandb: Currently logged in as: sbyim (use `wandb login --relogin` to force relogin)
wandb: wandb version 0.10.30 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Reduce to /job:localhost/replica:0/task:0/device:CPU:0 then broadcast to ('/job:localhost/replica:0/task:0/device:CPU:0',).
INFO:tensorflow:Redu

All model checkpoint layers were used when initializing TFRobertaForSequenceClassification.

Some layers of TFRobertaForSequenceClassification were not initialized from the model checkpoint at roberta-large and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1/10
learning rate: tf.Tensor(0.0, shape=(), dtype=float32)iteration: MirroredVariable:{
  0: <tf.Variable 'iter:0' shape=() dtype=int64, numpy=0>,
  1: <tf.Variable 'iter/replica_1:0' shape=() dtype=int64, numpy=0>,
  2: <tf.Variable 'iter/replica_2:0' shape=() dtype=int64, numpy=0>,
  3: <tf.Variable 'iter/replica_3:0' shape=() dtype=int64, numpy=0>,
  4: <tf.Variable 'iter/replica_4:0' shape=() dtype=int64, numpy=0>,
  5: <tf.Variable 'iter/replica_5:0' shape=() dtype=int64, numpy=0>,
  6: <tf.Variable 'iter/replica_6:0' shape=() dtype=int64, numpy=0>,
  7: <tf.Variable 'iter/replica_7:0' shape=() dtype=int64, numpy=0>
}
Please report this to the TensorFlow team. When filing the bug, set the verbosity to 10 (on Linux, `export AUTOGRAPH_VERBOSITY=10`) and attach the full output.
Cause: module, class, method, function, traceback, frame, or code object was expected, got cython_function_or_method
To silence this warning, decorate the function with @tf.autograph.experimental.do_not

wandb: ERROR Can't save model, h5py returned error: Saving the model to HDF5 format requires the model to be a Functional model or a Sequential model. It does not work for subclassed models, because such models are defined via the body of a Python method, which isn't safely serializable. Consider saving to the Tensorflow SavedModel format (by setting save_format="tf") or using `save_weights`.


Epoch 2/10
learning rate: tf.Tensor(4.998575e-06, shape=(), dtype=float32)iteration: MirroredVariable:{
  0: <tf.Variable 'iter:0' shape=() dtype=int64, numpy=391>,
  1: <tf.Variable 'iter/replica_1:0' shape=() dtype=int64, numpy=391>,
  2: <tf.Variable 'iter/replica_2:0' shape=() dtype=int64, numpy=391>,
  3: <tf.Variable 'iter/replica_3:0' shape=() dtype=int64, numpy=391>,
  4: <tf.Variable 'iter/replica_4:0' shape=() dtype=int64, numpy=391>,
  5: <tf.Variable 'iter/replica_5:0' shape=() dtype=int64, numpy=391>,
  6: <tf.Variable 'iter/replica_6:0' shape=() dtype=int64, numpy=391>,
  7: <tf.Variable 'iter/replica_7:0' shape=() dtype=int64, numpy=391>
}
391/391 [==============================] - 964s 2s/step - loss: 4.5529 - weighted_categorical_accuracy: 0.3885 - weighted_f1_score: 6.4686e-05 - val_loss: 0.3207 - val_weighted_categorical_accuracy: 0.1273 - val_weighted_f1_score: 4.5787e-05
Epoch 3/10
learning rate: tf.Tensor(4.441595e-06, shape=(), dtype=float32)iteration: MirroredVa

TypeError: float() argument must be a string or a number, not 'WarmUp'

In [ ]:
class_weights_dict

In [ ]:
model.config.id2label=labelMap

In [ ]:
# This part of the code utilized TFTrainer, doesn't work atm.
#from transformers import TFBertForSequenceClassification, TFTrainer, TFTrainingArguments
#training_args = TFTrainingArguments(
#    output_dir='./results',          # output directory
#    num_train_epochs=3,              # total # of training epochs
#    per_device_train_batch_size=16,  # batch size per device during training
#    per_device_eval_batch_size=16,   # batch size for evaluation
#    warmup_steps=500,                # number of warmup steps for learning rate scheduler
#    weight_decay=0.01,               # strength of weight decay
#    logging_dir='./logs',            # directory for storing logs
#)
#trainer = TFTrainer(
#    model=model,                         # the instantiated 🤗 Transformers model to be trained
#    args=training_args,                  # training arguments, defined above
#   train_dataset=train_dataset,    # tensorflow_datasets training dataset
#    eval_dataset=test_dataset       # tensorflow_datasets evaluation dataset
#)
#trainer.train()

In [ ]:
import matplotlib.pyplot as plt
history_dict = history.history
history_dict.keys()
acc = history_dict['weighted_categorical_accuracy']
val_acc = history_dict['val_weighted_categorical_accuracy']
loss = history_dict['loss']
val_loss = history_dict['val_loss']

epochs = range(1, len(acc) + 1)

# "bo" is for "blue dot"
plt.plot(epochs, loss, 'bo', label='Training loss')
# b is for "solid blue line"
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()

plt.show()

plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend(loc='lower right')

plt.show()

In [ ]:
model.summary()
test_result = model.evaluate(test_dataset.batch(32))
print(model.metrics_names)
test_result

In [ ]:
loss = CategoricalCrossentropy(from_logits=True)
metric = [CategoricalAccuracy(), F1Score(num_classes=5, average='weighted')]
optimizer = create_optimizer(init_lr=wandb.config.lr, weight_de)
#optimizer = AdamWeightDecay(learning_rate=wandb.config.lr, weight_decay_rate=wandb.config.weight_decay_rate)
loaded_model.compile(optimizer=optimizer, loss=loss, metrics=metric, weighted_metrics=metric)
best_result = loaded_model.evaluate(test_dataset.batch(32))
print(loaded_model.metrics_names)
best_result

In [ ]:
result

In [ ]:
test_dataset.element_spec
for text, label in test_dataset.take(3):
    print(text)
    print(label)

In [2]:
import os
save_path = os.path.join("mwsa","mwsa_best")



In [3]:
model.summary()
model.save_pretrained(save_path, saved_model=True)
tokenizer.save_pretrained(save_path)

NameError: name 'model' is not defined

In [ ]:
import transformers
transformers.logging.set_verbosity_info()
result = model.predict(test_dataset.batch(32))

In [ ]:
result.logits

In [ ]:
train_dataset?

In [ ]:
loaded

In [4]:
# In order to use GCP for trained models, we have to save the model as TF Saved_Model format.
# Just calling model.save() doesn't work.
# https://colab.research.google.com/drive/1p1Nifh1P-vqAZ1gHsNSCXAHzVWzl5YPP
# https://github.com/huggingface/transformers/issues/2021
import os 

from transformers import TFBertForSequenceClassification
loaded_model = TFBertForSequenceClassification.from_pretrained(save_path)


404 Client Error: Not Found for url: https://huggingface.co/mwsa/mwsa_best/resolve/main/config.json


OSError: Can't load config for 'mwsa/mwsa_best'. Make sure that:

- 'mwsa/mwsa_best' is a correct model identifier listed on 'https://huggingface.co/models'

- or 'mwsa/mwsa_best' is the correct path to a directory containing a config.json file



In [ ]:
loaded_model.evaluate(test_dataset.batch(16))

In [ ]:
print(test_dataset)

In [1]:
from transformers import BertTokenizerFast
import tensorflow as tf

loaded_tokenizer = BertTokenizerFast.from_pretrained(save_path)
input_encodings = loaded_tokenizer(["only me"],["only me"], max_length=max_length, truncation=True, padding='max_length', return_tensors="tf")

input_encodings


NameError: name 'save_path' is not defined

In [ ]:
data = tf.data.Dataset.from_tensor_slices((
    dict(input_encodings)
))

print(data)

for ids in input_encodings["input_ids"]:
  print(loaded_tokenizer.decode(ids))

In [ ]:
loaded_model.summary()

In [ ]:

#predict_input = (test_encodings,  truncation=True, max_length=128, padding='max_length')
result = loaded_model.predict(test_dataset.batch(32))
probabilities = tf.nn.softmax(result.logits)

In [ ]:
result.logits.shape

In [2]:
predict_input = (test_encodings,  truncation=True, max_length=max_length, padding='max_length')

SyntaxError: invalid syntax (<ipython-input-2-c91096c1418f>, line 1)

In [49]:
probabilities.shape

TensorShape([544, 5])

In [50]:
print(result.logits.shape) #(128*batchlength, num of labels)
print(result.logits) 


(544, 5)
[[-1.4968938  1.8233801  1.8028307 -2.201978  -1.6621366]
 [-1.3648084  2.5407703  1.3034997 -1.7457976 -1.8486904]
 [-0.9769792  1.4272964  2.137048  -2.0812771 -1.9733627]
 ...
 [-2.3338146  2.0951207  1.1570156 -0.8065253 -0.8023103]
 [-1.3284367  2.34798    1.6983447 -1.7060848 -1.7351846]
 [-1.3589617  3.1047041  0.8355413 -1.3977504 -1.0957118]]


In [38]:
#lst = range(5)
labels = le.classes_
labelMap = {v: k for v, k in enumerate(labels)}
    
print(labelMap)
#>>> {k: v for v, k in enumerate(lst)}
#{'A': 0, 'C': 2, 'B': 1}

{0: 'broader', 1: 'exact', 2: 'narrower', 3: 'none', 4: 'related'}


In [39]:
#import sys
#import numpy
!pip install scikit-learn
import sklearn
from sklearn.metrics import classification_report
numpy.set_printoptions(threshold=200)

preds = tf.argmax(result.logits, axis=1).numpy()
print(classification_report(np.argmax(test_labels,axis=1),preds))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00         3
           1       0.25      0.71      0.37        85
           2       0.06      0.17      0.09        29
           3       0.95      0.40      0.56       411
           4       0.02      0.06      0.04        16

    accuracy                           0.42       544
   macro avg       0.26      0.27      0.21       544
weighted avg       0.76      0.42      0.49       544



In [40]:
result.logits.shape

(544, 5)

In [55]:
result_df = pd.Series(preds)
result_df = result_df.map(lambda res: le.inverse_transform([res])[0])

In [56]:
test_data_orig[4]=result_df.values

In [59]:
test_data_orig.to_csv("bert_20210217", sep='\t', index = False)

In [ ]:
for ind, res in enumerate(preds):
    print(test_data.loc[[ind]])
    print(res)
    print(le.inverse_transform([res]))

In [ ]:

indices = list(range(0,test_data.shape[0]))

In [ ]:
index = 0
length = test_data.shape[0]

print(test_data.loc[[index]])
print('test_label' + str(test_labels[index]))
print(probabilities[index*128])
print('predicted: ' + str(le.inverse_transform([probabilities[index*128].numpy().argmax(axis=-1)])))
print(le.inverse_transform([0]))
print(le.inverse_transform([1]))
print(le.inverse_transform([2]))
print(le.inverse_transform([3]))
print(le.inverse_transform([4]))


In [ ]:
test_dataset

In [ ]:
!pip install pytorch


In [ ]:
from transformers import BertForSequenceClassification

pytorch_model = BertForSequenceClassification.from_pretrained(save_path, from_tf=True)
